#### Use CNN model to classify customized candlestick pattern
* candlestick_cnn_R08723060_Travis.py

In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

In [2]:
PARAMS = {}
PARAMS['pkl_name'] = './data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 10
PARAMS['batch_size'] = 64
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/10
235/235 - 3s - loss: 1.5586 - accuracy: 0.4135
Epoch 2/10
235/235 - 3s - loss: 0.7702 - accuracy: 0.7239
Epoch 3/10
235/235 - 3s - loss: 0.5900 - accuracy: 0.7847
Epoch 4/10
235/235 - 3s - loss: 0.5247 - accuracy: 0.8091
Epoch 5/10
235/235 - 3s - loss: 0.4887 - accuracy: 0.8212
Epoch 6/10
235/235 - 3s - loss: 0.4619 - accuracy: 0.8291
Epoch 7/10
235/235 - 4s - loss: 0.4377 - accuracy: 0.8411
Epoch 8/10
235/235 - 4s - loss: 0.4195 - accuracy: 0.8448
Epoch 9/10
235/235 - 3s - loss: 0.4119 - accuracy: 0.8488
Epoch 10/10
235/235 - 3s - loss: 0.3987 - accuracy: 0.8575
CNN test accuracy: 0.8185999989509583
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[2079  101  182  107  115   54   45  257   60]

#### 2. Use LSTM model to classify customized candlestick pattern
* candlestick_lstm_R087823060_Travis.py

In [3]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam

In [4]:
def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))
    
def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)
    
def mnist_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 150
    batch_size = 128

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 256
    n_classes = 10

    data = load_pkl('./data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print('epoch:{}'.format(training_iters))
    print_result(data, x_train, x_test, model)

In [5]:
mnist_lstm_main()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               304128    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 306,698
Trainable params: 306,698
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
118/118 [==============================] - 3s 28ms/step - loss: 2.0942 - accuracy: 0.2077 - val_loss: 1.7353 - val_accuracy: 0.2368
Epoch 2/150
118/118 [==============================] - 3s 26ms/step - loss: 1.4782 - accuracy: 0.3529 - val_loss: 1.4108 - val_accuracy: 0.3618
Epoch 3/150
118/118 [==============================] - 4s 32ms/step - loss: 1.3061 - accur